In [ ]:
%web_drop_table(WORK.Hotel_Reservation);


FILENAME REFFILE '/home/u62791171/Hotel_Reservations.csv';

PROC IMPORT DATAFILE=REFFILE
	DBMS=CSV
	OUT=WORK.Hotel_Reservation;
	GETNAMES=YES;
RUN;

PROC CONTENTS DATA=WORK.Hotel_Reservation; RUN;


%web_open_table(WORK.Hotel_Reservation);

/*-------------------------------------------------CODE-------------------------------------------------*/
/*Hotel Reservations:*/


/* Define Pie template */
proc template;
	define statgraph SASStudio.Pie;
		begingraph;
		entrytitle "Distribution of booking status" / textattrs=(size=14);
		layout region;
		piechart category=booking_status / stat=pct;
		endlayout;
		endgraph;
	end;
run;

ods graphics / reset width=6.4in height=4.8in imagemap;

proc sgrender template=SASStudio.Pie data=WORK.HOTEL_RESERVATION;
run;
/* Define Pie template */
proc template;
	define statgraph SASStudio.Pie;
		begingraph;
		entrytitle "Distribution of Meals types" / textattrs=(size=14);
		layout region;
		piechart category=type_of_meal_plan / stat=pct;
		endlayout;
		endgraph;
	end;
run;

ods graphics / reset width=6.4in height=4.8in imagemap;

proc sgrender template=SASStudio.Pie data=WORK.HOTEL_RESERVATION;
run;

ods graphics / reset;
ods graphics / reset;
/*What are the features of the hotel reservation data? (e.g. hotel name, date, room type, customer information, etc.)*/
data first5;
   set Hotel_Reservation (obs=5);
run;
/*What is the size of the dataset (number of rows and columns)*/
proc means data = Hotel_Reservation;
run;
/*What is the target variable in the dataset*/
proc means data=Hotel_Reservation;
   var avg_price_per_room;
run;
/*What is the distribution of target variable*/

PROC TEMPLATE;
   DEFINE STATGRAPH pie;
      BEGINGRAPH;
         LAYOUT REGION;
            PIECHART CATEGORY =  booking_status
            DATALABELLOCATION = INSIDE
            DATALABELCONTENT = ALL
            CATEGORYDIRECTION = CLOCKWISE
            DATASKIN = SHEEN 
            START = 180 NAME = 'pie';
            DISCRETELEGEND 'pie' /
            TITLE = 'Booking Status';
         ENDLAYOUT;
      ENDGRAPH;
   END;
RUN;
PROC SGRENDER DATA = Hotel_Reservation
            TEMPLATE = pie;
RUN;

/*How does the customer information (e.g. age, gender, nationality) affect hotel reservations*/

/*What is the average length of stay for hotel customers*/
proc means data=Hotel_Reservation;
   var no_of_weekend_nights;
run;

proc means data=Hotel_Reservation;
   var no_of_week_nights;
run;

/*Year of booking by Month*/
proc SGPLOT data = Hotel_Reservation;
vbar arrival_year /group = arrival_month ;
title 'Year of booking by Month';
run;
quit;

/*What is the distribution of reservations across different hotels and room types*/
PROC TEMPLATE;
   DEFINE STATGRAPH pie;
      BEGINGRAPH;
         LAYOUT REGION;
            PIECHART CATEGORY = room_type_reserved /
            DATALABELLOCATION = INSIDE
            DATALABELCONTENT = ALL
            CATEGORYDIRECTION = CLOCKWISE
            DATASKIN = SHEEN 
            START = 180 NAME = 'pie';
            DISCRETELEGEND 'pie' /
            TITLE = 'Room Types';
         ENDLAYOUT;
      ENDGRAPH;
   END;
RUN;
PROC SGRENDER DATA = Hotel_Reservation
            TEMPLATE = pie;
RUN;

/*What is the average daily rate (ADR) and revenue per available room (RevPAR) for each hotel*/
title "Average price and total revenue per hotel";
proc sgplot data=Hotel_Reservation;
    styleattrs datacontrastcolors=(red green blue);
    series x=arrival_year y=avg_price_per_room / group=room_type_reserved;
run;
title;

proc SGPLOT data = Hotel_Reservation;
vbar arrival_year /group = room_type_reserved  GROUPDISPLAY = CLUSTER;
title 'Average price and total revenue per hotel';
run;
quit;

/*How does the occupancy rate vary across different months and weekdays*/
proc SGPLOT data = Hotel_Reservation;
vbar arrival_month /group = room_type_reserved  GROUPDISPLAY = CLUSTER;
title 'Average price and total revenue per hotel';
run;
quit;

proc SGPLOT data = Hotel_Reservation;
vbar arrival_day /group = room_type_reserved  GROUPDISPLAY = CLUSTER;
title 'Average price and total revenue per hotel';
run;
quit;

/* Define Pie template */
proc template;
	define statgraph SASStudio.Pie;
		begingraph;
		entrytitle "Distribution of market segment" / textattrs=(size=14);
		layout region;
		piechart category=market_segment_type / stat=pct;
		endlayout;
		endgraph;
	end;
run;

ods graphics / reset width=6.4in height=4.8in imagemap;

proc sgrender template=SASStudio.Pie data=WORK.HOTEL_RESERVATION;
run;

ods graphics / reset;

/*repeated guest*/

ods graphics / reset width=6.4in height=4.8in imagemap;

proc sgplot data=WORK.HOTEL_RESERVATION;
	vbar booking_status / response=repeated_guest fillattrs=(color=CXff82ff);
	yaxis grid;
run;

ods graphics / reset;

proc sort data=WORK.HOTEL_RESERVATION out=_BarLineChartTaskData;
	by booking_status;
run;

/* Compute axis ranges */
proc means data=_BarLineChartTaskData noprint;
	by booking_status;
	class type_of_meal_plan / order=data;
	var no_of_children no_of_adults;
	output out=_BarLine_(where=(_type_ > 0)) mean(no_of_children 
		no_of_adults)=resp1 resp2;
run;

/* Compute response min and max values (include 0 in computations) */
data _null_;
	retain respmin 0 respmax 0;
	retain respmin1 0 respmax1 0 respmin2 0 respmax2 0;
	set _BarLine_ end=last;
	respmin1=min(respmin1, resp1);
	respmin2=min(respmin2, resp2);
	respmax1=max(respmax1, resp1);
	respmax2=max(respmax2, resp2);

	if last then
		do;
			call symputx ("respmin1", respmin1);
			call symputx ("respmax1", respmax1);
			call symputx ("respmin2", respmin2);
			call symputx ("respmax2", respmax2);
			call symputx ("respmin", min(respmin1, respmin2));
			call symputx ("respmax", max(respmax1, respmax2));
		end;
run;

/* Define a macro for offset */
%macro offset ();
	%if %sysevalf(&respmin eq 0) %then
		%do;
			offsetmin=0 %end;

	%if %sysevalf(&respmax eq 0) %then
		%do;
			offsetmax=0 %end;
%mend offset;

ods graphics / reset width=6.4in height=4.8in imagemap;

proc sgplot data=_BarLineChartTaskData nocycleattrs;
	by booking_status;
	vbar type_of_meal_plan / response=no_of_children fillattrs=(color=CXad85ff) 
		stat=mean;
	vline type_of_meal_plan / response=no_of_adults stat=mean y2axis;
	yaxis grid min=&respmin1 max=&respmax1 %offset();
	y2axis min=&respmin2 max=&respmax2 %offset();
	keylegend / location=outside;
run;

ods graphics / reset;

proc datasets library=WORK noprint;
	delete _BarLine_ _BarLineChartTaskData;
	run;


proc sort data=WORK.HOTEL_RESERVATION out=_BarLineChartTaskData;
	by booking_status;
run;

/* Compute axis ranges */
proc means data=_BarLineChartTaskData noprint;
	by booking_status;
	class type_of_meal_plan / order=data;
	var no_of_children no_of_adults;
	output out=_BarLine_(where=(_type_ > 0)) mean(no_of_children 
		no_of_adults)=resp1 resp2;
run;

/* Compute response min and max values (include 0 in computations) */
data _null_;
	retain respmin 0 respmax 0;
	retain respmin1 0 respmax1 0 respmin2 0 respmax2 0;
	set _BarLine_ end=last;
	respmin1=min(respmin1, resp1);
	respmin2=min(respmin2, resp2);
	respmax1=max(respmax1, resp1);
	respmax2=max(respmax2, resp2);

	if last then
		do;
			call symputx ("respmin1", respmin1);
			call symputx ("respmax1", respmax1);
			call symputx ("respmin2", respmin2);
			call symputx ("respmax2", respmax2);
			call symputx ("respmin", min(respmin1, respmin2));
			call symputx ("respmax", max(respmax1, respmax2));
		end;
run;

/* Define a macro for offset */
%macro offset ();
	%if %sysevalf(&respmin eq 0) %then
		%do;
			offsetmin=0 %end;

	%if %sysevalf(&respmax eq 0) %then
		%do;
			offsetmax=0 %end;
%mend offset;

ods graphics / reset width=6.4in height=4.8in imagemap;

proc sgplot data=_BarLineChartTaskData nocycleattrs;
	by booking_status;
	vbar type_of_meal_plan / response=no_of_children fillattrs=(color=CXad85ff) 
		stat=mean;
	vline type_of_meal_plan / response=no_of_adults stat=mean y2axis;
	yaxis grid min=&respmin1 max=&respmax1 %offset();
	y2axis min=&respmin2 max=&respmax2 %offset();
	keylegend / location=outside;
run;

ods graphics / reset;

proc datasets library=WORK noprint;
	delete _BarLine_ _BarLineChartTaskData;
	run;